# Preservation potential grids

This notebook converts the cumulative erosion values at the grid point locations to true netCDF format grids of preservation potential, using the parametric distribution fitted in notebook `04`.

If training data has been extracted from the source datasets by running the `00c-extract_training_data_global.ipynb` and `00b-extract_grid_data.ipynb` notebooks, set the `use_extracted_data` variable below to `True` to use this dataset instead of the pre-prepared training data from the [Zenodo repository](https://zenodo.org/record/8157691).

## Notebook options

These cells set some of the important variables and definitions used throughout the notebook.

In [1]:
# If True, use training data extracted in notebook 00c
# Else, use pre-prepared training data downloaded from Zenodo
use_extracted_data = True

# Number of processes to use
n_jobs = 4

# Control verbosity level of logging output
verbose = False

These values will usually not need to be modified:

In [ ]:
from lib.check_files import check_prepared_data

if use_extracted_data:
    data_dir = "extracted_data"
else:
    data_dir = check_prepared_data("prepared_data")

outputs_dir = "outputs"

If any of the following exist as environment variables, they will replace the values defined above:

In [ ]:
import os

n_jobs = int(os.environ.get("N_JOBS", n_jobs))
verbose = bool(int(os.environ.get("VERBOSE", verbose)))

## Notebook setup

Imports, definitions, etc.

### Imports

In [2]:
import glob

import joblib
import numpy as np
import pandas as pd
from gplately import Raster

from lib.pu import create_grids

### Input and output files

In [4]:
data_filename = os.path.join(data_dir, "grid_data.csv")

outputs_dir = "outputs"
dist_parametric_filename = os.path.join(
    outputs_dir,
    "global",
    "erodep_dist.joblib",
)

# Output
output_filename_points = os.path.join(
    outputs_dir,
    "global",
    "grid_data_preservation_likelihood.csv",
)
erodep_grid_dir = os.path.join(
    outputs_dir,
    "erosion_grids",
)
likelihood_dir = os.path.join(
    outputs_dir,
    "global",
    "preservation_likelihood_grids",
)
for i in (
    erodep_grid_dir,
    likelihood_dir,
):
    os.makedirs(i, exist_ok=True)

### Load data and calculate likelihood

The likelihood point values will be saved to a `.csv` file in the output directory.

In [5]:
cols = ["lon", "lat", "age (Ma)", "erosion (m)"]
data = pd.read_csv(data_filename, usecols=cols)

if verbose:
    print(
        "Loading parametric distribution from file: "
        + dist_parametric_filename
    )
dist_parametric = joblib.load(dist_parametric_filename)
data["likelihood"] = dist_parametric.pdf(data["erosion (m)"])

data.to_csv(output_filename_points, index=False)

Loading parametric distribution from file: outputs/global/erodep_dist_parametric.joblib


Loading KDE distribution from file: outputs/global/erodep_dist_kde.joblib


## Create grids

In this section, the different types of output grids are created.

### Erosion

In [6]:
create_grids(
    data,
    output_dir=erodep_grid_dir,
    threads=n_jobs,
    verbose=verbose,
    column="erosion (m)",
    filename_format="erosion_grid_{}Ma.nc",
    extent="global",
)


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   28.7s


[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   53.6s


[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:   57.1s


[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  1.8min


...

[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:  6.5min


[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:  7.6min


[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:  9.4min


[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed: 11.4min


[Parallel(n_jobs=8)]: Done 171 out of 171 | elapsed: 13.8min finished


### Preservation likelihood

In [7]:
for erodep_filename in glob.glob(
    os.path.join(
        erodep_grid_dir,
        r"erosion_grid_*Ma.nc",
    )
):
    if verbose:
        print(
            f"Reading file: {os.path.basename(erodep_filename)}"
        )
    likelihood_basename = os.path.basename(
        erodep_filename
    ).replace("erosion_grid", "preservation_likelihood_grid")
    likelihood_filename = os.path.join(
        likelihood_dir,
        likelihood_basename,
    )

    erodep_raster = Raster(erodep_filename)

    likelihood = Raster(dist_parametric.pdf(erodep_raster))
    if verbose:
        print(
            f" - Writing likelihood file: {likelihood_basename}"
        )
    likelihood.save_to_netcdf4(likelihood_filename)

Reading file: erosion_grid_32Ma.nc
 - Writing parametric file: preservation_likelihood_grid_32Ma.nc


 - Writing KDE file: preservation_likelihood_grid_32Ma.nc
Reading file: erosion_grid_111Ma.nc
 - Writing parametric file: preservation_likelihood_grid_111Ma.nc


 - Writing KDE file: preservation_likelihood_grid_111Ma.nc
Reading file: erosion_grid_146Ma.nc
 - Writing parametric file: preservation_likelihood_grid_146Ma.nc


 - Writing KDE file: preservation_likelihood_grid_146Ma.nc
Reading file: erosion_grid_65Ma.nc
 - Writing parametric file: preservation_likelihood_grid_65Ma.nc


 - Writing KDE file: preservation_likelihood_grid_65Ma.nc
Reading file: erosion_grid_73Ma.nc
 - Writing parametric file: preservation_likelihood_grid_73Ma.nc


...

 - Writing KDE file: preservation_likelihood_grid_70Ma.nc
Reading file: erosion_grid_27Ma.nc
 - Writing parametric file: preservation_likelihood_grid_27Ma.nc


 - Writing KDE file: preservation_likelihood_grid_27Ma.nc
Reading file: erosion_grid_104Ma.nc
 - Writing parametric file: preservation_likelihood_grid_104Ma.nc


 - Writing KDE file: preservation_likelihood_grid_104Ma.nc
Reading file: erosion_grid_128Ma.nc
 - Writing parametric file: preservation_likelihood_grid_128Ma.nc


 - Writing KDE file: preservation_likelihood_grid_128Ma.nc
Reading file: erosion_grid_169Ma.nc
 - Writing parametric file: preservation_likelihood_grid_169Ma.nc


 - Writing KDE file: preservation_likelihood_grid_169Ma.nc
